#### Задание 1
Добавьте к таблице следующие поля:

* Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
* Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
* Стоимость последнего проданного дома до текущего (1 балл)

#### Задание 2
В итоге у вас таблица с колонками (или нечто похожее):

* price
* Среднегодовая цена
* Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
* Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
* Стоимость последнего проданного дома до текущего ((1 балл)
* и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)). (2 балла)

#### Задание 3
SQL like case when или if elif else

Создайте колонку, в которой в которой будет отображаться "+", "-" или "=", если "Средняя стомость 10 проданных домов до текущего в том же районе" больше, меньше или равно "Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)", соотвественно.

Если одно из полей Null, запишите в эту колонку "Нет данных"


In [44]:
from pyspark.sql import SparkSession, Window, functions as F
from pyspark.sql.types import *

spark = SparkSession.builder \
    .master("local[4]") \
    .appName("Lesson_4") \
    .config("spark.executor.instances", 2) \
    .config("spark.executor.memory", f'{int(2000/4.4)}mb') \
    .config("spark.executor.cores", 2) \
    .getOrCreate()

sc = spark.sparkContext

In [45]:
# читаем данные
data = spark.read.csv('raw_sales.csv', header=True, inferSchema=True)

In [46]:
data.show(truncate=False)

+-------------------+--------+-------+------------+--------+
|datesold           |postcode|price  |propertyType|bedrooms|
+-------------------+--------+-------+------------+--------+
|2007-02-07 00:00:00|2607    |525000 |house       |4       |
|2007-02-27 00:00:00|2906    |290000 |house       |3       |
|2007-03-07 00:00:00|2905    |328000 |house       |3       |
|2007-03-09 00:00:00|2905    |380000 |house       |4       |
|2007-03-21 00:00:00|2906    |310000 |house       |3       |
|2007-04-04 00:00:00|2905    |465000 |house       |4       |
|2007-04-24 00:00:00|2607    |399000 |house       |3       |
|2007-04-30 00:00:00|2606    |1530000|house       |4       |
|2007-05-24 00:00:00|2902    |359000 |house       |3       |
|2007-05-25 00:00:00|2906    |320000 |house       |3       |
|2007-06-26 00:00:00|2902    |385000 |house       |3       |
|2007-06-27 00:00:00|2906    |305000 |house       |3       |
|2007-06-27 00:00:00|2612    |850000 |house       |4       |
|2007-06-28 00:00:00|290

### Задание 1

In [47]:
windAvgTo = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold')\
    .rowsBetween(Window.currentRow - 10, Window.currentRow)

windAvgAfter = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold')\
    .rowsBetween(Window.currentRow, Window.currentRow + 10)

windDatesoldLag = Window()\
    .orderBy('datesold')\
    .rowsBetween(Window.unboundedPreceding, Window.currentRow - 1)

df = data.withColumn('avg_to', F.avg('price').over(windAvgTo))\
         .withColumn('avg_after', F.avg('price').over(windAvgAfter))\
         .withColumn('DatesoldLag', F.last('datesold').over(windDatesoldLag))

In [48]:
df.show()

+-------------------+--------+-------+------------+--------+-----------------+------------------+-------------------+
|           datesold|postcode|  price|propertyType|bedrooms|           avg_to|         avg_after|        DatesoldLag|
+-------------------+--------+-------+------------+--------+-----------------+------------------+-------------------+
|2007-02-07 00:00:00|    2607| 525000|       house|       4|         525000.0| 559313.6363636364|               null|
|2007-02-27 00:00:00|    2906| 290000|       house|       3|         290000.0| 363772.7272727273|2007-02-07 00:00:00|
|2007-03-07 00:00:00|    2905| 328000|       house|       3|         328000.0|393454.54545454547|2007-02-27 00:00:00|
|2007-03-09 00:00:00|    2905| 380000|       house|       4|         354000.0| 394090.9090909091|2007-03-07 00:00:00|
|2007-03-21 00:00:00|    2906| 310000|       house|       3|         300000.0| 393772.7272727273|2007-03-09 00:00:00|
|2007-04-04 00:00:00|    2905| 465000|       house|     

### Задание 2

In [49]:
df.withColumn('uniq_vals', F.size(F.array_distinct(F.array(*df.columns)))).show()

+-------------------+--------+-------+------------+--------+-----------------+------------------+-------------------+---------+
|           datesold|postcode|  price|propertyType|bedrooms|           avg_to|         avg_after|        DatesoldLag|uniq_vals|
+-------------------+--------+-------+------------+--------+-----------------+------------------+-------------------+---------+
|2007-02-07 00:00:00|    2607| 525000|       house|       4|         525000.0| 559313.6363636364|               null|        8|
|2007-02-27 00:00:00|    2906| 290000|       house|       3|         290000.0| 363772.7272727273|2007-02-07 00:00:00|        8|
|2007-03-07 00:00:00|    2905| 328000|       house|       3|         328000.0|393454.54545454547|2007-02-27 00:00:00|        8|
|2007-03-09 00:00:00|    2905| 380000|       house|       4|         354000.0| 394090.9090909091|2007-03-07 00:00:00|        8|
|2007-03-21 00:00:00|    2906| 310000|       house|       3|         300000.0| 393772.7272727273|2007-03

### Задание 3

In [50]:
df = df.withColumn('is_null', F.col("avg_to").isNull() | F.col("avg_after").isNull())

def f(x, y, condition):
    if condition:
        return 'Нет данных'
    else:
        if x > y:
            return '+'
        elif x < y:
            return '-'
        else:
            return '='

udf_f = F.udf(f, returnType=StringType())

df.withColumn('f_val', udf_f(df.avg_to, df.avg_after, df.is_null)).show()

+-------------------+--------+-------+------------+--------+-----------------+------------------+-------------------+-------+-----+
|           datesold|postcode|  price|propertyType|bedrooms|           avg_to|         avg_after|        DatesoldLag|is_null|f_val|
+-------------------+--------+-------+------------+--------+-----------------+------------------+-------------------+-------+-----+
|2007-02-07 00:00:00|    2607| 525000|       house|       4|         525000.0| 559313.6363636364|               null|  false|    -|
|2007-02-27 00:00:00|    2906| 290000|       house|       3|         290000.0| 363772.7272727273|2007-02-07 00:00:00|  false|    -|
|2007-03-07 00:00:00|    2905| 328000|       house|       3|         328000.0|393454.54545454547|2007-02-27 00:00:00|  false|    -|
|2007-03-09 00:00:00|    2905| 380000|       house|       4|         354000.0| 394090.9090909091|2007-03-07 00:00:00|  false|    -|
|2007-03-21 00:00:00|    2906| 310000|       house|       3|         300000.